# Aligning non-manuscript source to target using Engine versification



## Prerequisites

Clone the following to the same directory.

```
git clone https://github.com/Clear-Bible/ClearEngine
git clone https://github.com/russellmorley/machine.git
```

In [1]:
#r "nuget:SIL.Scripture,10.1.0"
#r "nuget:Thot,3.3.5"
#r "../src/ClearBible.Engine/bin/Debug/net7.0/ClearBible.Engine.dll"
#r "../src/ClearBible.Engine.SyntaxTree.Aligner/bin/Debug/net7.0/ClearBible.Engine.SyntaxTree.Aligner.dll"   
#r "../../machine/src/SIL.Machine/bin/Debug/netstandard2.0/Clear.SIL.Machine.dll"
#r "../../machine/src/SIL.Machine.Translation.Thot/bin/Debug/netstandard2.0/SIL.Machine.Translation.Thot.dll"

using SIL.Machine.Corpora;
using SIL.Machine.Tokenization;
using SIL.Machine.Translation;
using SIL.Machine.Translation.Thot;
using static SIL.Machine.Corpora.TokenProcessors;
using SIL.Machine.Utils;

using ClearBible.Engine.Corpora;
using ClearBible.Engine.Exceptions;
using ClearBible.Engine.SyntaxTree.Aligner.Persistence;
using ClearBible.Engine.SyntaxTree.Aligner.Translation;
using ClearBible.Engine.Tokenization;
using ClearBible.Engine.Translation;
using ClearBible.Engine.Persistence;

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages SIL.Scripture, 10.1.0 Thot, 3.3.5

## Prepare corpora

In [2]:
var sourceCorpus = new ParatextTextCorpus("data/VBL-PT");
var sourceCorpusTransformed = sourceCorpus
    .Tokenize<LatinWordTokenizer>()
    .Transform<IntoTokensTextRowProcessor>()
    .Transform<SetTrainingBySurfaceLowercase>();

var targetCorpus = new ParatextTextCorpus("data/WEB-PT");
var targetCorpusTransformed = targetCorpus
    .Tokenize<LatinWordTokenizer>()
    .Transform<IntoTokensTextRowProcessor>()
    .Transform<SetTrainingBySurfaceLowercase>();

var parallelTextCorpus = sourceCorpusTransformed.EngineAlignRows(
    targetCorpusTransformed
    ,new SourceTextIdToVerseMappingsFromVerseMappings(
    EngineParallelTextCorpus.VerseMappingsForAllVerses(sourceCorpus.Versification, targetCorpus.Versification))
);



// Helper function for detokenizing text.
public static void WriteTokensEngineParallelTextRow(
    EngineParallelTextRow engineParallelTextRow, 
    IDetokenizer<IEnumerable<(Token token, string paddingBefore, string paddingAfter)>, Token> sourceDetokenizer, 
    IDetokenizer<IEnumerable<(Token token, string paddingBefore, string paddingAfter)>, Token> targetDetokenizer)
{
    Console.WriteLine(engineParallelTextRow.Ref.ToString());

    //SOURCE

    //TRAINING:
    //Token ids for segments (training) and training text, which is used to build segments:
    var tokenIdsForSegmentsAndTrainingText = engineParallelTextRow.SourceTokens!
        .Select(t => t.TokenId);
    Console.WriteLine($"Source segments tokenIds      : {string.Join(" ", tokenIdsForSegmentsAndTrainingText)}");

    //segment text, used by training
    Console.WriteLine($"Source segments spaced        : {string.Join(" ", engineParallelTextRow.SourceSegment)}");

    //training text, used to build segments
    var trainingTexts = engineParallelTextRow.SourceTokens!
        .Select(t => t.TrainingText);
    Console.WriteLine($"Source trainingText spaced    : {string.Join(" ", trainingTexts)}");

    //DISPLAY:
    //Token ids for surface text (display):
    var tokenIdsForSurfaceText = engineParallelTextRow.SourceTokens!
        .GetPositionalSortedBaseTokens() //pull out the tokens from composite tokens
        .Select(t => t.TokenId);
    Console.WriteLine($"Source tokens tokenIds        : {string.Join(" ", tokenIdsForSurfaceText)}");

    //Surface text, still tokenized
    var surfaceTexts = engineParallelTextRow.SourceTokens!
        .GetPositionalSortedBaseTokens()
        .Select(t => t.SurfaceText);
    Console.WriteLine($"Source surfaceTexts spaced    : {string.Join(" ", surfaceTexts)}");

    //Surface text, detokenized
    var tokensWithPadding = sourceDetokenizer.Detokenize(engineParallelTextRow.SourceTokens!.GetPositionalSortedBaseTokens());
    Console.WriteLine($"Source detokenized surfaceText: {tokensWithPadding.Aggregate(string.Empty, (constructedString, tokenWithPadding) => $"{constructedString}{tokenWithPadding.paddingBefore}{tokenWithPadding.token}{tokenWithPadding.paddingAfter}")}");
    Console.WriteLine("");


    //TARGET

    //TRAINING:
    //Token ids for segments (training) and training text, which is used to build segments:
    tokenIdsForSegmentsAndTrainingText = engineParallelTextRow.TargetTokens!
        .Select(t => t.TokenId);
    Console.WriteLine($"Target segments tokenIds      : {string.Join(" ", tokenIdsForSegmentsAndTrainingText)}");

    //segment text, used by training
    Console.WriteLine($"Target segments spaced        : {string.Join(" ", engineParallelTextRow.TargetSegment)}");

    //training text, used to build segments
    trainingTexts = engineParallelTextRow.TargetTokens!
        .Select(t => t.TrainingText);
    Console.WriteLine($"Target trainingText spaced    : {string.Join(" ", trainingTexts)}");

    //DISPLAY:
    //Token ids for surface text (display):
    tokenIdsForSurfaceText = engineParallelTextRow.TargetTokens!
        .GetPositionalSortedBaseTokens() //pull out the tokens from composite tokens
        .Select(t => t.TokenId);
    Console.WriteLine($"Target tokens tokenIds        : {string.Join(" ", tokenIdsForSurfaceText)}");

    //Surface text, still tokenized
    surfaceTexts = engineParallelTextRow.TargetTokens!
        .GetPositionalSortedBaseTokens()
        .Select(t => t.SurfaceText);
    Console.WriteLine($"Target surfaceTexts spaced    : {string.Join(" ", surfaceTexts)}");

    //Surface text, detokenized
    tokensWithPadding = targetDetokenizer.Detokenize(engineParallelTextRow.TargetTokens!.GetPositionalSortedBaseTokens());
    Console.WriteLine($"Target detokenized surfaceText: {tokensWithPadding.Aggregate(string.Empty, (constructedString, tokenWithPadding) => $"{constructedString}{tokenWithPadding.paddingBefore}{tokenWithPadding.token}{tokenWithPadding.paddingAfter}")}");
    Console.WriteLine("");
}

## Approach 1: save model then load it to get alignments

In [3]:
using System.IO;

Directory.CreateDirectory("out/VBL-WEB-FASTALIGN");
{
    using var srcTrgTrainer = new ThotWordAlignmentModelTrainer(ThotWordAlignmentModelType.FastAlign, parallelTextCorpus.Lowercase(),
        "out/VBL-WEB-FASTALIGN/src_trg");

    using var trgSrcTrainer = new ThotWordAlignmentModelTrainer(ThotWordAlignmentModelType.FastAlign,
        parallelTextCorpus.Invert().Lowercase(), "out/VBL-WEB-FASTALIGN/trg_src");

    using var symmetrizedTrainer = new SymmetrizedWordAlignmentModelTrainer(srcTrgTrainer, trgSrcTrainer);

    symmetrizedTrainer.Train(new DelegateProgress(status =>
        Console.WriteLine($"{status.Message}: {status.PercentCompleted:P}")));
    symmetrizedTrainer.Save();
 
    Console.WriteLine("Symmetrized FastAlign model saved");
}

Training direct alignment model: 0.00%
Training direct alignment model: 0.00%
Training direct alignment model: 8.33%
Training direct alignment model: 16.67%
Training direct alignment model: 25.00%
Training direct alignment model: 33.33%
Training direct alignment model: 41.67%
Training direct alignment model: 50.00%
Training inverse alignment model: 50.00%
Training inverse alignment model: 50.00%
Training inverse alignment model: 58.33%
Training inverse alignment model: 66.67%
Training inverse alignment model: 75.00%
Training inverse alignment model: 83.33%
Training inverse alignment model: 91.67%
Training inverse alignment model: 100.00%
Symmetrized FastAlign model saved


In [4]:
{
    using var srcTrgModel = new ThotFastAlignWordAlignmentModel("out/VBL-WEB-FASTALIGN/src_trg");
    using var trgSrcModel = new ThotFastAlignWordAlignmentModel("out/VBL-WEB-FASTALIGN/trg_src");
    using var smtWordAlignmentModel = new SymmetrizedWordAlignmentModel(srcTrgModel, trgSrcModel)
    {
        Heuristic = SymmetrizationHeuristic.GrowDiagFinalAnd
    };
    
    foreach (var engineParallelTextRow in parallelTextCorpus.Cast<EngineParallelTextRow>())
    {
        WriteTokensEngineParallelTextRow(engineParallelTextRow, new EngineStringDetokenizer(new WhitespaceDetokenizer()), new EngineStringDetokenizer(new LatinWordDetokenizer()));

        //predict primary smt aligner alignments only then display - ONLY FOR COMPARISON
        var smtOrdinalAlignments = smtWordAlignmentModel.GetBestAlignment(engineParallelTextRow.SourceSegment, engineParallelTextRow.TargetSegment);
        IEnumerable<AlignedTokenPairs> smtSourceTargetTokenIdPairs = engineParallelTextRow.GetAlignedTokenPairs(smtOrdinalAlignments);
        // (Legacy): Alignments as ordinal positions in versesmap
        Console.WriteLine($"SMT Alignment        : {smtOrdinalAlignments}");
        // Alignments as source token to target token pairs
        Console.WriteLine($"SMT Alignment        : {string.Join(" ", smtSourceTargetTokenIdPairs.Select(t => $"{t.SourceToken.TokenId}->{t.TargetToken.TokenId}"))}");
    }
}

1JN 1:1
Source segments tokenIds      : 062001001001001 062001001002001 062001001003001 062001001004001 062001001005001 062001001006001 062001001007001 062001001008001 062001001009001 062001001010001 062001001011001 062001001012001 062001001013001 062001001014001 062001001015001 062001001016001 062001001017001 062001001018001 062001001019001 062001001020001 062001001021001 062001001022001 062001001023001 062001001024001 062001001025001 062001001026001 062001001027001 062001001028001 062001001029001 062001001030001 062001001031001 062001001032001 062001001033001 062001001034001 062001001035001 062001001036001 062001001037001 062001001038001
Source segments spaced        : esta carta trata sobre la palabra de vida que existía desde el principio , que hemos escuchado , que hemos visto con nuestros propios ojos y le hemos contemplado , y que hemos tocado con nuestras manos .
Source trainingText spaced    : esta carta trata sobre la palabra de vida que existía desde el principio , que hemos

Target segments spaced        : ( and the life was revealed , and we have seen , and testify , and declare to you the life , the eternal life , which was with the father , and was revealed to us ) ;
Target trainingText spaced    : ( and the life was revealed , and we have seen , and testify , and declare to you the life , the eternal life , which was with the father , and was revealed to us ) ;
Target tokens tokenIds        : 062001002001001 062001002002001 062001002003001 062001002004001 062001002005001 062001002006001 062001002007001 062001002008001 062001002009001 062001002010001 062001002011001 062001002012001 062001002013001 062001002014001 062001002015001 062001002016001 062001002017001 062001002018001 062001002019001 062001002020001 062001002021001 062001002022001 062001002023001 062001002024001 062001002025001 062001002026001 062001002027001 062001002028001 062001002029001 062001002030001 062001002031001 062001002032001 062001002033001 062001002034001 062001002035001 0620010020

1JN 1:4
Source segments tokenIds      : 062001004001001 062001004002001 062001004003001 062001004004001 062001004005001 062001004006001 062001004007001 062001004008001 062001004009001 062001004010001 062001004011001 062001004012001 062001004013001 062001004014001
Source segments spaced        : escribimos para decirles esto , a fin de que nuestra felicidad sea completa .
Source trainingText spaced    : escribimos para decirles esto , a fin de que nuestra felicidad sea completa .
Source tokens tokenIds        : 062001004001001 062001004002001 062001004003001 062001004004001 062001004005001 062001004006001 062001004007001 062001004008001 062001004009001 062001004010001 062001004011001 062001004012001 062001004013001 062001004014001
Source surfaceTexts spaced    : Escribimos para decirles esto , a fin de que nuestra felicidad sea completa .
Source detokenized surfaceText: Escribimos para decirles esto , a fin de que nuestra felicidad sea completa .

Target segments tokenIds      : 0620010

Source surfaceTexts spaced    : Si decimos ser sus amigos , y seguimos viviendo en oscuridad , estamos mintiendo , y no vivimos en la verdad .
Source detokenized surfaceText: Si decimos ser sus amigos , y seguimos viviendo en oscuridad , estamos mintiendo , y no vivimos en la verdad .

Target segments tokenIds      : 062001006001001 062001006002001 062001006003001 062001006004001 062001006005001 062001006006001 062001006007001 062001006008001 062001006009001 062001006010001 062001006011001 062001006012001 062001006013001 062001006014001 062001006015001 062001006016001 062001006017001 062001006018001 062001006019001 062001006020001 062001006021001 062001006022001 062001006023001
Target segments spaced        : if we say that we have fellowship with him and walk in the darkness , we lie and don’t tell the truth .
Target trainingText spaced    : if we say that we have fellowship with him and walk in the darkness , we lie and don’t tell the truth .
Target tokens tokenIds        : 062001006

Source segments spaced        : si decimos que no pecamos , nos engañamos a nosotros mismos , y la verdad no está en nosotros .
Source trainingText spaced    : si decimos que no pecamos , nos engañamos a nosotros mismos , y la verdad no está en nosotros .
Source tokens tokenIds        : 062001008001001 062001008002001 062001008003001 062001008004001 062001008005001 062001008006001 062001008007001 062001008008001 062001008009001 062001008010001 062001008011001 062001008012001 062001008013001 062001008014001 062001008015001 062001008016001 062001008017001 062001008018001 062001008019001 062001008020001
Source surfaceTexts spaced    : Si decimos que no pecamos , nos engañamos a nosotros mismos , y la verdad no está en nosotros .
Source detokenized surfaceText: Si decimos que no pecamos , nos engañamos a nosotros mismos , y la verdad no está en nosotros .

Target segments tokenIds      : 062001008001001 062001008002001 062001008003001 062001008004001 062001008005001 062001008006001 0620010

...

3JN 1:7
Source segments tokenIds      : 064001007001001 064001007002001 064001007003001 064001007004001 064001007005001 064001007006001 064001007007001 064001007008001 064001007009001 064001007010001 064001007011001 064001007012001 064001007013001 064001007014001 064001007015001 064001007016001 064001007017001
Source segments spaced        : pues viajan en su nombre , y no aceptan nada de los que no son creyentes .
Source trainingText spaced    : pues viajan en su nombre , y no aceptan nada de los que no son creyentes .
Source tokens tokenIds        : 064001007001001 064001007002001 064001007003001 064001007004001 064001007005001 064001007006001 064001007007001 064001007008001 064001007009001 064001007010001 064001007011001 064001007012001 064001007013001 064001007014001 064001007015001 064001007016001 064001007017001
Source surfaceTexts spaced    : pues viajan en su nombre , y no aceptan nada de los que no son creyentes .
Source detokenized surfaceText: pues viajan en su nombre , y no


SMT Alignment        : 0-1 1-1 2-1 3-2 4-2 5-3 6-4 7-5 8-6 9-7 10-8 11-9 12-9 11-10 11-11 18-11 17-12 15-13 16-13 20-16 21-17 23-17 22-18 24-19 25-19 23-20 26-21 27-22
SMT Alignment        : 064001009001001->064001009002001 064001009002001->064001009002001 064001009003001->064001009002001 064001009004001->064001009003001 064001009005001->064001009003001 064001009006001->064001009004001 064001009007001->064001009005001 064001009008001->064001009006001 064001009009001->064001009007001 064001009010001->064001009008001 064001009011001->064001009009001 064001009012001->064001009010001 064001009013001->064001009010001 064001009012001->064001009011001 064001009012001->064001009012001 064001009019001->064001009012001 064001009018001->064001009013001 064001009016001->064001009014001 064001009017001->064001009014001 064001009021001->064001009017001 064001009022001->064001009018001 064001009024001->064001009018001 064001009023001->064001009019001 064001009025001->064001009020001 064001009026001-

3JN 1:11
Source segments tokenIds      : 064001011001001 064001011002001 064001011003001 064001011004001 064001011005001 064001011006001 064001011007001 064001011008001 064001011009001 064001011010001 064001011011001 064001011012001 064001011013001 064001011014001 064001011015001 064001011016001 064001011017001 064001011018001 064001011019001 064001011020001 064001011021001 064001011022001 064001011023001 064001011024001 064001011025001 064001011026001 064001011027001 064001011028001 064001011029001 064001011030001 064001011031001
Source segments spaced        : amigo mío , no imites el mal , sino el bien . los que hacen el bien le pertenecen a dios ; los que hacen el mal no lo conocen .
Source trainingText spaced    : amigo mío , no imites el mal , sino el bien . los que hacen el bien le pertenecen a dios ; los que hacen el mal no lo conocen .
Source tokens tokenIds        : 064001011001001 064001011002001 064001011003001 064001011004001 064001011005001 064001011006001 064001011007001


SMT Alignment        : 1-0 2-3 4-3 3-4 3-5 5-6 6-6 7-7 8-9 9-10 11-11 12-11 13-12 15-14 14-15 16-15 17-16 20-17 22-18 23-19 24-20 25-21 25-22 26-22 27-22 26-23 28-24 28-25 29-26 30-27
SMT Alignment        : 064001012002001->064001012001001 064001012003001->064001012004001 064001012005001->064001012004001 064001012004001->064001012005001 064001012004001->064001012006001 064001012006001->064001012007001 064001012007001->064001012007001 064001012008001->064001012008001 064001012009001->064001012010001 064001012010001->064001012011001 064001012012001->064001012012001 064001012013001->064001012012001 064001012014001->064001012013001 064001012016001->064001012015001 064001012015001->064001012016001 064001012017001->064001012016001 064001012018001->064001012017001 064001012021001->064001012018001 064001012023001->064001012019001 064001012024001->064001012020001 064001012025001->064001012021001 064001012026001->064001012022001 064001012026001->064001012023001 064001012027001->064001012023001 

Target detokenized surfaceText: but I hope to see you soon. Then we will speak face to face. Peace be to you. The friends greet you. Greet the friends by name.

SMT Alignment        : 0-2 1-2 2-2 3-3 4-3 5-4 10-4 7-5 8-6 9-7 11-9 13-10 14-12 15-12 16-12 15-13 15-14 16-15 12-16 19-17 27-19 26-20 28-20 25-21 30-22 31-23 32-24 33-27 39-28 34-29 35-30 42-31
SMT Alignment        : 064001014001001->064001014003001 064001014002001->064001014003001 064001014003001->064001014003001 064001014004001->064001014004001 064001014005001->064001014004001 064001014006001->064001014005001 064001014011001->064001014005001 064001014008001->064001014006001 064001014009001->064001014007001 064001014010001->064001014008001 064001014012001->064001014010001 064001014014001->064001014011001 064001014015001->064001014013001 064001014016001->064001014013001 064001014017001->064001014013001 064001014016001->064001014014001 064001014016001->064001014015001 064001014017001->064001014016001 064001014013001->0640010140

## Approach 2: don't save model and get alignments

In [5]:
{
    using var srcTrgModel = new ThotFastAlignWordAlignmentModel();
    //using var trainerSrcTrg = srcTrgModel.CreateTrainer(parallelTextCorpusList, TokenProcessors.Lowercase, TokenProcessors.Lowercase);

    using var trgSrcModel = new ThotFastAlignWordAlignmentModel();
    //using var trainerTrgSrc = trgSrcModel.CreateTrainer(parallelTextCorpusList.Invert(), TokenProcessors.Lowercase, TokenProcessors.Lowercase);

    
    using var smtWordAlignmentModel = new SymmetrizedWordAlignmentModel(srcTrgModel, trgSrcModel)
    {
        Heuristic = SymmetrizationHeuristic.GrowDiagFinalAnd
    };
    using var smtWordAlignmentModeltrainer = smtWordAlignmentModel.CreateTrainer(parallelTextCorpus.Lowercase());
    smtWordAlignmentModeltrainer.Train(new DelegateProgress(status =>
        Console.WriteLine($"Training symmetrized fast align model: {status.PercentCompleted:P}")));
    smtWordAlignmentModeltrainer.Save();
    
    foreach (var engineParallelTextRow in parallelTextCorpus.Cast<EngineParallelTextRow>())
    {
        WriteTokensEngineParallelTextRow(engineParallelTextRow, new EngineStringDetokenizer(new WhitespaceDetokenizer()), new EngineStringDetokenizer(new LatinWordDetokenizer()));

        //predict primary smt aligner alignments only then display - ONLY FOR COMPARISON
        var smtOrdinalAlignments = smtWordAlignmentModel.GetBestAlignment(engineParallelTextRow.SourceSegment, engineParallelTextRow.TargetSegment);
        IEnumerable<AlignedTokenPairs> smtSourceTargetTokenIdPairs = engineParallelTextRow.GetAlignedTokenPairs(smtOrdinalAlignments);
        // (Legacy): Alignments as ordinal positions in versesmap
        Console.WriteLine($"SMT Alignment        : {smtOrdinalAlignments}");
        // Alignments as source token to target token pairs
        Console.WriteLine($"SMT Alignment        : {string.Join(" ", smtSourceTargetTokenIdPairs.Select(t => $"{t.SourceToken.TokenId}->{t.TargetToken.TokenId}"))}");
   }
}


Training symmetrized fast align model: 0.00%
Training symmetrized fast align model: 0.00%
Training symmetrized fast align model: 8.33%
Training symmetrized fast align model: 16.67%
Training symmetrized fast align model: 25.00%
Training symmetrized fast align model: 33.33%
Training symmetrized fast align model: 41.67%
Training symmetrized fast align model: 50.00%
Training symmetrized fast align model: 50.00%
Training symmetrized fast align model: 50.00%
Training symmetrized fast align model: 58.33%
Training symmetrized fast align model: 66.67%
Training symmetrized fast align model: 75.00%
Training symmetrized fast align model: 83.33%
Training symmetrized fast align model: 91.67%
Training symmetrized fast align model: 100.00%
1JN 1:1
Source segments tokenIds      : 062001001001001 062001001002001 062001001003001 062001001004001 062001001005001 062001001006001 062001001007001 062001001008001 062001001009001 062001001010001 062001001011001 062001001012001 062001001013001 062001001014001 06

Source detokenized surfaceText: Esta Vida nos fue revelada . La vimos y damos testimonio de ella . Estamos hablándoles de Aquél que es la Vida Eterna , que estaba con el Padre , y que nos fue revelado .

Target segments tokenIds      : 062001002001001 062001002002001 062001002003001 062001002004001 062001002005001 062001002006001 062001002007001 062001002008001 062001002009001 062001002010001 062001002011001 062001002012001 062001002013001 062001002014001 062001002015001 062001002016001 062001002017001 062001002018001 062001002019001 062001002020001 062001002021001 062001002022001 062001002023001 062001002024001 062001002025001 062001002026001 062001002027001 062001002028001 062001002029001 062001002030001 062001002031001 062001002032001 062001002033001 062001002034001 062001002035001 062001002036001 062001002037001 062001002038001 062001002039001
Target segments spaced        : ( and the life was revealed , and we have seen , and testify , and declare to you the life , the eternal lif

1JN 1:4
Source segments tokenIds      : 062001004001001 062001004002001 062001004003001 062001004004001 062001004005001 062001004006001 062001004007001 062001004008001 062001004009001 062001004010001 062001004011001 062001004012001 062001004013001 062001004014001
Source segments spaced        : escribimos para decirles esto , a fin de que nuestra felicidad sea completa .
Source trainingText spaced    : escribimos para decirles esto , a fin de que nuestra felicidad sea completa .
Source tokens tokenIds        : 062001004001001 062001004002001 062001004003001 062001004004001 062001004005001 062001004006001 062001004007001 062001004008001 062001004009001 062001004010001 062001004011001 062001004012001 062001004013001 062001004014001
Source surfaceTexts spaced    : Escribimos para decirles esto , a fin de que nuestra felicidad sea completa .
Source detokenized surfaceText: Escribimos para decirles esto , a fin de que nuestra felicidad sea completa .

Target segments tokenIds      : 0620010

Source surfaceTexts spaced    : Si decimos ser sus amigos , y seguimos viviendo en oscuridad , estamos mintiendo , y no vivimos en la verdad .
Source detokenized surfaceText: Si decimos ser sus amigos , y seguimos viviendo en oscuridad , estamos mintiendo , y no vivimos en la verdad .

Target segments tokenIds      : 062001006001001 062001006002001 062001006003001 062001006004001 062001006005001 062001006006001 062001006007001 062001006008001 062001006009001 062001006010001 062001006011001 062001006012001 062001006013001 062001006014001 062001006015001 062001006016001 062001006017001 062001006018001 062001006019001 062001006020001 062001006021001 062001006022001 062001006023001
Target segments spaced        : if we say that we have fellowship with him and walk in the darkness , we lie and don’t tell the truth .
Target trainingText spaced    : if we say that we have fellowship with him and walk in the darkness , we lie and don’t tell the truth .
Target tokens tokenIds        : 062001006

Source segments spaced        : si decimos que no pecamos , nos engañamos a nosotros mismos , y la verdad no está en nosotros .
Source trainingText spaced    : si decimos que no pecamos , nos engañamos a nosotros mismos , y la verdad no está en nosotros .
Source tokens tokenIds        : 062001008001001 062001008002001 062001008003001 062001008004001 062001008005001 062001008006001 062001008007001 062001008008001 062001008009001 062001008010001 062001008011001 062001008012001 062001008013001 062001008014001 062001008015001 062001008016001 062001008017001 062001008018001 062001008019001 062001008020001
Source surfaceTexts spaced    : Si decimos que no pecamos , nos engañamos a nosotros mismos , y la verdad no está en nosotros .
Source detokenized surfaceText: Si decimos que no pecamos , nos engañamos a nosotros mismos , y la verdad no está en nosotros .

Target segments tokenIds      : 062001008001001 062001008002001 062001008003001 062001008004001 062001008005001 062001008006001 0620010

...

3JN 1:7
Source segments tokenIds      : 064001007001001 064001007002001 064001007003001 064001007004001 064001007005001 064001007006001 064001007007001 064001007008001 064001007009001 064001007010001 064001007011001 064001007012001 064001007013001 064001007014001 064001007015001 064001007016001 064001007017001
Source segments spaced        : pues viajan en su nombre , y no aceptan nada de los que no son creyentes .
Source trainingText spaced    : pues viajan en su nombre , y no aceptan nada de los que no son creyentes .
Source tokens tokenIds        : 064001007001001 064001007002001 064001007003001 064001007004001 064001007005001 064001007006001 064001007007001 064001007008001 064001007009001 064001007010001 064001007011001 064001007012001 064001007013001 064001007014001 064001007015001 064001007016001 064001007017001
Source surfaceTexts spaced    : pues viajan en su nombre , y no aceptan nada de los que no son creyentes .
Source detokenized surfaceText: pues viajan en su nombre , y no


SMT Alignment        : 0-1 1-1 2-1 3-2 4-2 5-3 6-4 7-5 8-6 9-7 10-8 11-9 12-9 11-10 11-11 18-11 17-12 15-13 16-13 20-16 21-17 23-17 22-18 24-19 25-19 23-20 26-21 27-22
SMT Alignment        : 064001009001001->064001009002001 064001009002001->064001009002001 064001009003001->064001009002001 064001009004001->064001009003001 064001009005001->064001009003001 064001009006001->064001009004001 064001009007001->064001009005001 064001009008001->064001009006001 064001009009001->064001009007001 064001009010001->064001009008001 064001009011001->064001009009001 064001009012001->064001009010001 064001009013001->064001009010001 064001009012001->064001009011001 064001009012001->064001009012001 064001009019001->064001009012001 064001009018001->064001009013001 064001009016001->064001009014001 064001009017001->064001009014001 064001009021001->064001009017001 064001009022001->064001009018001 064001009024001->064001009018001 064001009023001->064001009019001 064001009025001->064001009020001 064001009026001-

3JN 1:11
Source segments tokenIds      : 064001011001001 064001011002001 064001011003001 064001011004001 064001011005001 064001011006001 064001011007001 064001011008001 064001011009001 064001011010001 064001011011001 064001011012001 064001011013001 064001011014001 064001011015001 064001011016001 064001011017001 064001011018001 064001011019001 064001011020001 064001011021001 064001011022001 064001011023001 064001011024001 064001011025001 064001011026001 064001011027001 064001011028001 064001011029001 064001011030001 064001011031001
Source segments spaced        : amigo mío , no imites el mal , sino el bien . los que hacen el bien le pertenecen a dios ; los que hacen el mal no lo conocen .
Source trainingText spaced    : amigo mío , no imites el mal , sino el bien . los que hacen el bien le pertenecen a dios ; los que hacen el mal no lo conocen .
Source tokens tokenIds        : 064001011001001 064001011002001 064001011003001 064001011004001 064001011005001 064001011006001 064001011007001


SMT Alignment        : 1-0 2-3 4-3 3-4 3-5 5-6 6-6 7-7 8-9 9-10 11-11 12-11 13-12 15-14 14-15 16-15 17-16 20-17 22-18 23-19 24-20 25-21 25-22 26-22 27-22 26-23 28-24 28-25 29-26 30-27
SMT Alignment        : 064001012002001->064001012001001 064001012003001->064001012004001 064001012005001->064001012004001 064001012004001->064001012005001 064001012004001->064001012006001 064001012006001->064001012007001 064001012007001->064001012007001 064001012008001->064001012008001 064001012009001->064001012010001 064001012010001->064001012011001 064001012012001->064001012012001 064001012013001->064001012012001 064001012014001->064001012013001 064001012016001->064001012015001 064001012015001->064001012016001 064001012017001->064001012016001 064001012018001->064001012017001 064001012021001->064001012018001 064001012023001->064001012019001 064001012024001->064001012020001 064001012025001->064001012021001 064001012026001->064001012022001 064001012026001->064001012023001 064001012027001->064001012023001 

Target detokenized surfaceText: but I hope to see you soon. Then we will speak face to face. Peace be to you. The friends greet you. Greet the friends by name.

SMT Alignment        : 0-2 1-2 2-2 3-3 4-3 5-4 10-4 7-5 8-6 9-7 11-9 13-10 14-12 15-12 16-12 15-13 15-14 16-15 12-16 19-17 27-19 26-20 28-20 25-21 30-22 31-23 32-24 33-27 39-28 34-29 35-30 42-31
SMT Alignment        : 064001014001001->064001014003001 064001014002001->064001014003001 064001014003001->064001014003001 064001014004001->064001014004001 064001014005001->064001014004001 064001014006001->064001014005001 064001014011001->064001014005001 064001014008001->064001014006001 064001014009001->064001014007001 064001014010001->064001014008001 064001014012001->064001014010001 064001014014001->064001014011001 064001014015001->064001014013001 064001014016001->064001014013001 064001014017001->064001014013001 064001014016001->064001014014001 064001014016001->064001014015001 064001014017001->064001014016001 064001014013001->0640010140